# C/C++ Library with Python 

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. 

But for certain types of calculations Python (and any other interpreted language) can be **very slow**.

Such calculations may be implemented in a compiled language such as C or Fortran.

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

### 1.1 Compiler a `shared library` with `multiple` files

The shared library with `multiple` code files

* funs.c/h

* SumArray.c/h

In [ ]:
%%file ./demo/src/funs.h

#ifndef FUNS_H
#define FUNS_H

double dprod(double *x, int n);

#endif

In [ ]:
%%file ./demo/src/funs.c

#include "funs.h"

// x[0]*x[1]*...*x[n-1]
double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

#### Building `funs.c`  and `SumArray.c` into libmultifuns.dll

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/funs.o ./demo/src/funs.c 
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/SumArray.o  ./demo/src/SumArray.c
!gcc -shared -o ./demo/bin/libmultifuns.dll  ./demo/obj/funs.o  ./demo/obj/SumArray.o

In [ ]:
!dir .\demo\bin\libmulti*.dll

#### Building with makefile

In [ ]:
%%file ./demo/makefile-libmultifun


CC=gcc
CFLAGS=-O3 -Wall -fPIC


SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC) -shared -o $(BINDIR)libmultifuns.dll $(OBJDIR)funs.o $(OBJDIR)SumArray.o
	 del .\demo\obj\funs.o .\demo\obj\SumArray.o
    
multifunsobj: $(SRCDIR)funs.c $(SRCDIR)SumArray.c
	$(CC) -c $(CFLAGS) -o $(OBJDIR)SumArray.o $(SRCDIR)SumArray.c
	$(CC) -c $(CFLAGS) -o $(OBJDIR)funs.o $(SRCDIR)funs.c 
     
     
clean:
	 del .\demo\bin\libmultifuns.dll

In [ ]:
!make -f ./demo/makefile-libmultifun

The result is a compiled shared library **`libmultifuns.dll`**

#####  makefile-libmultifun Using new variables

In [ ]:
%%file ./code/makefile-libmultifun

CC=gcc
CFLAGS=-O3 -Wall -fPIC  

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

INC = -I$(SRCDIR) 

SRCS= $(SRCDIR)funs.c \
      $(SRCDIR)SumArray.c 

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC)  -shared -o $(BINDIR)libmultifuns.dll funs.o SumArray.o
	 del funs.o SumArray.o
    
multifunsobj: 
	 $(CC) -c $(CFLAGS) $(INC) $(SRCS) 
     
clean:
	 del .\demo\bin\libmultifuns.dll

In [ ]:
!make -f ./code/makefile-libmultifun

##### call the Lib

In [ ]:
%%file ./demo/src/mainMultifuns.c

#include <stdio.h> 
#include "SumArray.h"
#include "funs.h"

int main() {
    
     int a1[] = {8, 4, 5, 3, 2};
     printf("sum is %d\n", sum(a1, 5));  // sum is 22
    
     double a2[] = {8.0, 4.0, 5.0, 3.0, 2.0};
     printf("dprod is %f\n", dprod(a2, 5));  // dprod is 960
     return 0;
}

In [ ]:
!gcc -c -o ./demo/obj/mainMultifuns.o ./demo/src/mainMultifuns.c 
!gcc -o  ./demo/bin/mainMultifuns ./demo/obj/mainMultifuns.o -I./demo/src/ -L./demo/bin/ -lmultifuns

In [ ]:
!.\demo\bin\mainMultifuns

### 1.2 ctypes - access the C library

**ctypes** is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

http://docs.python.org/3/library/ctypes.html

We need to load the library and set properties such as the functions return and argument types using the **ctypes** package :

* **ctypes** exports the **cdll**, and on Windows **windll** objects, for loading dynamic link libraries.
   * **cdll.LoadLibrary(name)** : loads the library  which export functions using standard `__cdecl` calling convention
   * **windll.LoadLibrary(name)** : loads the library with `__stdcall` calling convention for the function 


* **argtypes** : the types of the arguments 


* **restype**: the types of return values. 

#### Example `__cdecl` calling convention for the function(windows,Linux)

**msvcrt** is the MS **standard C library** containing most standard C functions, and uses the cdecl calling convention:

In [ ]:
from ctypes import *
print(cdll.msvcrt)

#### Example `__stdcall` calling convention for the function(windows only) 

In [ ]:
from ctypes import *
print(windll.kernel32)  

### Wrap libmultifuns.dll in pure Python.

`__cdecl` calling convention

In [5]:
%%file ./demo/src/multifuns.py

from ctypes import cdll,c_int,c_double,POINTER

_lib = cdll.LoadLibrary('./demo/bin/libmultifuns.dll')

# double dprod(double *x, int n)
def dprod(x):
    _lib.dprod.argtypes = [POINTER(c_double), c_int]
    _lib.dprod.restype  = c_double
    n = len(x)
    #  convert a Python list into a C array by using ctypes
    arr= (c_double * n)(*x)
    return _lib.dprod(arr,int(n))

# int isum(int array[], int size);
def isum(x):
    _lib.sum.argtypes = [POINTER(c_int), c_int]
    _lib.sum.restype =c_int
    n = len(x)
    #  convert a Python list into a C array by using ctypes
    arr= (c_int * n)(*x)
    return _lib.sum(arr,int(n))

Overwriting ./demo/src/multifuns.py


In [3]:
import sys
sys.path.append('./demo/src')

In [6]:
import multifuns
print(multifuns.dprod([8.0, 4.0, 5.0, 3.0, 2.0])) 
print(multifuns.isum([8, 4, 5, 3, 2]))

960.0
22


### Further reading

ctypes http://docs.python.org/3/library/ctypes.html

C-Types Foreign Function Interface (numpy.ctypeslib)
  https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html
  